Este es mi proyecto de Pipelines

Your code should be saved in a Python executable file (.py), your data should be saved in a folder named data, and your results should be saved in a folder named output.

You should also include a README.md file that describes the steps you took and your thought process as you built your data pipeline.

In [7]:
#La base de datos con las que vamos a trabajar es la base Muestral de Voto Nulo para el Estado de Aguscalientes.
#Procederemos a cargarla

import pandas as pd

In [8]:
#DATA ACQUISITION

#Cargamos la dirección y la guardamos como un data frame
direccion = "C:/Users/Ramiro Antonio/Documents/ironhack_labs/pipelines-project/your-code/Voto_Nulo_Ags.csv"
voto_nulo = pd.read_csv(direccion, index_col=0) 

In [11]:
voto_nulo.head(10)

Distrito  Municipio  Seccion Tipo_secc  Edad Sexo Casilla  Voto  \
Estado                                                                    
1              1          2      338         M    61    H      C1     2   
1              1          2      338         M    50    H      C1     1   
1              1          2      338         M    31    H      C1     2   
1              1          2      338         M    23    H      C1     1   
1              1          2      338         M    36    M      C1     1   
1              1          2      338         M    33    M      C1     1   
1              1          2      338         M    31    M      C1     2   
1              1          2      338         M    46    M      C1     1   
1              1          2      338         M    36    M      C1     1   
1              1          2      338         M    33    M      C1     2   

        Ponderador_OK  
Estado                 
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526  
1            4.917526

In [100]:
voto_nulo.describe()

#Vemos que de la siguiente tabla que el promedio de edad del votante en Aguascalienes es 40 años.
#Hubo votante con hasta 103 años.

Distrito      Municipio        Seccion           Edad  \
count  181335.000000  181335.000000  181335.000000  181335.000000   
mean        1.905804       2.691703     312.617393      40.451656   
std         0.757476       2.848105     168.818704      16.286880   
min         1.000000       1.000000      10.000000      18.000000   
25%         1.000000       1.000000     147.000000      27.000000   
50%         2.000000       1.000000     328.000000      38.000000   
75%         2.000000       4.000000     445.000000      51.000000   
max         3.000000      11.000000     603.000000     103.000000   

                Voto  
count  181335.000000  
mean        1.427027  
std         0.494648  
min         1.000000  
25%         1.000000  
50%         1.000000  
75%         2.000000  
max         2.000000

In [10]:
#DATA WRANGLING

#Esta base de datos no dice el nombre del distrito electoral, sólo el ID. Eso no da información, por lo que incluímos los nombres
voto_nulo["Nombre_Distrito"] = ["José María" if x == 1 else "Aguascalientes" if x ==2 else "Hidroaguas" for x in voto_nulo["Distrito"]]
voto_nulo.head(5)


{'José María', 'Aguascalientes', 'Hidroaguas'}


Distrito  Municipio  Seccion Tipo_secc  Edad Sexo Casilla  Voto  \
Estado                                                                    
1              1          2      338         M    61    H      C1     2   
1              1          2      338         M    50    H      C1     1   
1              1          2      338         M    31    H      C1     2   
1              1          2      338         M    23    H      C1     1   
1              1          2      338         M    36    M      C1     1   

        Ponderador_OK Nombre_Distrito  
Estado                                 
1            4.917526      José María  
1            4.917526      José María  
1            4.917526      José María  
1            4.917526      José María  
1            4.917526      José María

In [13]:
#La variable de Ponderador_OK no nos interesa, por lo que la tiramos
voto_nulo = voto_nulo.drop(["Ponderador_OK"], axis = 1)

In [12]:
#Tipos de casillas. 
print(set(voto_nulo['Casilla']))


{'C4', 'E1 C4', 'C3A', 'C10', 'E1', 'C7', 'E1 C2', 'B', 'E1 C7', 'E1 C1', 'E1 C12', 'E1 C17', 'E1 C3', 'E1 C5', 'C8', 'E1 C9', 'E2 C2', 'E1 C8', 'E2', 'C3', 'E1 C13', 'C6', 'C2', 'C5', 'C9', 'C1'}


In [97]:
#Queremos eliminar Casillas Especiales (E1) que están solas, dado que no tienen padrón electoral predeterminado y sesgan análisis
voto_nulo = voto_nulo[voto_nulo.Casilla != "E1"]
print(set(voto_nulo['Casilla']))

{'C4', 'E1 C4', 'C3A', 'C10', 'C7', 'E1 C2', 'B', 'E1 C7', 'E1 C1', 'E1 C12', 'E1 C17', 'E1 C3', 'E1 C5', 'C8', 'E1 C9', 'E2 C2', 'E1 C8', 'E2', 'C3', 'E1 C13', 'C6', 'C2', 'C5', 'C9', 'C1'}


In [15]:
#Codificar variable Voto

#Se verifica si la "Voto" tiene valores anómalos (sólo debería haber dos valores)
print(set(voto_nulo["Voto"]))

{1, 2}


In [16]:
#En este caso sólo se tienen dos tipos de votos. Sin embargo, en caso hacemos una función para eliminar las valores que no 
#sean 1 (Votó) y 2 (No votó)

def LimpiarValoresVoto(df):
    #if (df.Voto != 1) or (df.Voto != 2):
    #   df = df[df.Voto == 1]
    #df.drop(df[(df['Voto'] != 1) and (df['Voto'] != 2)].index)
    #df.drop(df[(df['Voto'] != 1) and (df['Voto'] != 2)].index)
    #df = df.drop(df[df['Voto'] > 2])
    #df = df[df["Voto"].ge(3)]
    df = df[df["Voto"] <= 2]
    return pd.DataFrame(df)

print(LimpiarValoresVoto(voto_nulo))

        Distrito  Municipio  Seccion Tipo_secc  Edad Sexo Casilla  Voto  \
Estado                                                                    
1              1          2      338         M    61    H      C1     2   
1              1          2      338         M    50    H      C1     1   
1              1          2      338         M    31    H      C1     2   
1              1          2      338         M    23    H      C1     1   
1              1          2      338         M    36    M      C1     1   
1              1          2      338         M    33    M      C1     1   
1              1          2      338         M    31    M      C1     2   
1              1          2      338         M    46    M      C1     1   
1              1          2      338         M    36    M      C1     1   
1              1          2      338         M    33    M      C1     2   
1              1          2      338         M    32    M      C1     2   
1              1         

In [98]:
#Actualmente la variable "Voto" tiene dos valores: 1 y 2. Para facilitar la codificación se desea
#Integrar el código 1 = Votó, 2 = No votó.

#Para hacerlo, vamos a crear una nueva variable que se llame Voto_NoCodificado
voto_nulo["Voto_NoCodificado"] = ["Voto" if x == 1 else "No_voto" for x in voto_nulo["Voto"]]
voto_nulo.head(3)

Distrito Nombre_Distrito  Municipio  Seccion Tipo_secc  Edad Sexo  \
Estado                                                                      
1              1       josemaria          2      338         M    61    H   
1              1       josemaria          2      338         M    50    H   
1              1       josemaria          2      338         M    31    H   

       Casilla  Voto Voto_NoCodificado  
Estado                                  
1           C1     2           No_voto  
1           C1     1              Voto  
1           C1     2           No_voto

In [18]:
#Orden de columnas
voto_nulo.columns

voto_nulo = voto_nulo[['Distrito', 'Nombre_Distrito', 'Municipio', 'Seccion', 'Tipo_secc', 'Edad', 'Sexo',
       'Casilla', 'Voto', 'Voto_NoCodificado']]

In [19]:
#String operations

#vamos a pasar todas las mayuscúlas a minúsculas y eliminar los acentos y espacios en la variable "Nombre_Distrito"

voto_nulo["Nombre_Distrito"] = voto_nulo.Nombre_Distrito.str.lower()
voto_nulo["Voto_NoCodificado"] = voto_nulo.Voto_NoCodificado.str.lower()

In [20]:
#Quitar el espacio
voto_nulo["Nombre_Distrito"] = voto_nulo['Nombre_Distrito'].str.replace(" ","")
voto_nulo["Nombre_Distrito"] = voto_nulo['Nombre_Distrito'].str.replace("é","e")
voto_nulo["Nombre_Distrito"] = voto_nulo['Nombre_Distrito'].str.replace("í","i")

print(set(voto_nulo["Nombre_Distrito"]))

{'aguascalientes', 'hidroaguas', 'josemaria'}


In [96]:
#Data Analysis

def Número_de_registros_de_votantes(df):
    return df.count(axis = 0) 

print(Número_de_registros_de_votantes(voto_nulo))


Distrito             181335
Nombre_Distrito      181335
Municipio            181335
Seccion              181335
Tipo_secc            181335
Edad                 181335
Sexo                 181335
Casilla              181335
Voto                 181335
Voto_NoCodificado    181335
dtype: int64


In [58]:
def PersonasQueVotaron(df):
    return df.loc[df.Voto == 1, 'Voto'].count()

print(f'La cantidad de votos registrados es {PersonasQueVotaron(voto_nulo)}')


La cantidad de votos registrados es 103900


In [63]:
def Porcentaje_PersonasQueVotaron(df):
    return (df.loc[df.Voto == 1, 'Voto'].count())/len(df)*100

print(f'El porcentaje de participación en la Entidad Federativa es {Porcentaje_PersonasQueVotaron(voto_nulo)}')


El porcentaje de participación en la Entidad Federativa es 57.297267488350286


In [99]:
#Comparación con el promedio nacional. La participación nacional fue de 63.43%.
def Comparacion_ParticipacionNacional(df):
    if (df.loc[df.Voto == 1, 'Voto'].count())/len(df)*100 < 63.43:
        print("En los distritos de esta entidad federativa hubo menor participación que en el promedio nacional")
    else:
        return "En los distritos de esta entidad federativa hubo mayor particiación que en el promedio nacional"

print(Comparacion_ParticipacionNacional(voto_nulo))

En los distritos de esta entidad federativa hubo menor participación que en el promedio nacional
None


In [62]:
#Para saber los votos por sección.

def voto_por_seccion(df):
    df_temp= df[df.Voto == 1]
    urbanas = len([x for x in df_temp["Tipo_secc"] if x == "U"])
    rurales = len([x for x in df_temp["Tipo_secc"] if x == "R"])
    mixtas = len([x for x in df_temp["Tipo_secc"] if x == "M"])
    return f'En las secciones urbanas hubo {urbanas} votos. \n En las mixtas hubo {mixtas} votos. \n En las rurales {rurales} votos.'

print(voto_por_seccion(voto_nulo))

En las secciones urbanas hubo 67004 votos. 
 En las mixtas hubo 25416 votos. 
 En las rurales, 11480 votos.


In [102]:
#Comparación participación mujeres  y hombres
def batalla_sexos2(df):
    df_temp= df[df.Voto == 1] #Guarda todos los que votaron (antes de df había puesto voto_nulo)
    if df_temp.loc[df_temp.Sexo == "M", "Sexo"].count() > df_temp.loc[df_temp.Sexo == "H", "Sexo"].count():
        print(f'Votaron más mujeres. El porcentaje de mujeres que votó fue 
              {df_temp.loc[df_temp.Sexo == "M", "Sexo"].count()/len(df_temp)*100}')
    else:
        return f'Votaron más hombres. El porcentaje de hombres que votó fue 
              {df_temp.loc[df_temp.Sexo == "H", "Sexo"].count()/len(df_temp)*100}'

print(batalla_sexos2(voto_nulo))

Votaron más mujeres. El porcentaje de mujeres que votó fue 55.644850818094326
None
